# * Ex-T Revenue & Subs

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM (
        SELECT TM_KEY_DAY
            , CASE 	WHEN METRIC_CD LIKE 'TB%S%' 
                    THEN ROW_NUMBER() OVER (PARTITION BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY DESC)
                    ELSE 1 END FLAG
            , CASE WHEN METRIC_CD LIKE 'TB%S%' THEN 'SUB' ELSE 'REV' END AGG_TYPE
            , METRIC_CD, METRIC_NAME, ACTUAL, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK, SRC
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
        WHERE REMARK IN ('salisa', 'jutar11')
        AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
        AND EXISTS (
            SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
            FROM (
                SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            ) TMP
            WHERE TMP_CD = AREA_CD )
        AND TM_KEY_DAY >= {v_start_date} 
    ) TMP
    WHERE FLAG = 1
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-27, 10:01:40

   -> Summany DataFrame : 427 rows, 17 columns

TDMDBPR : Disconnected


In [3]:
''' Current Month Revenue '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_overall_df = chk_src_df.copy()
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_overall_df = src_t_rev_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_overall_df = src_t_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_overall_df[col] = src_t_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-26 04:44:33,20250301,20250323,0,0,"1,853,638,095","1,853,629,953","1,853,629,953","1,845,870,444",0,5,1104,25392
1,TB2R000100,Postpaid Revenue,2025-03-26 04:20:14,20250302,20250325,0,"2,925,621,638","2,925,484,550","2,925,482,756","2,925,482,756",0,0,5,177,1246
2,TB2R010100,Postpaid Revenue B2C,2025-03-26 04:21:45,20250302,20250325,0,"2,650,614,876","2,650,561,958","2,650,560,164","2,650,560,164","2,641,896,715",0,6,1105,7744
3,TB2R020100,Postpaid Revenue B2B,2025-03-26 04:21:10,20250302,20250322,0,"275,006,762","274,922,591","274,922,591","274,922,591",0,0,5,177,1239
4,TB3R000100,Broadband Revenue,2025-03-26 05:07:05,20250302,20250322,0,0,"1,272,708,395","1,272,708,395","1,272,708,395","1,272,801,517",0,5,1102,7709
5,TB4R000100,TVS Revenue,2025-03-19 14:24:58,20250302,20250316,0,0,"117,600,632","117,877,263","117,925,721",0,0,4,176,880


In [4]:
''' Average Revenue per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_avg_mth_df = chk_src_df.copy()
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_avg_mth_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_avg_mth_df[col] = src_t_rev_avg_mth_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_avg_mth_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-20 01:42:42,20240101,20250228,0,0,"2,331,116,542","2,331,084,640","2,331,084,640","2,321,354,989",0,5,1104,469200
1,TB2R000100,Postpaid Revenue,2025-03-19 11:39:10,20240102,20250228,0,"3,702,523,438","3,671,427,102","3,671,411,395","3,671,411,395",0,0,5,177,24603
2,TB2R010100,Postpaid Revenue B2C,2025-03-19 11:40:02,20240102,20250228,0,"3,293,619,861","3,287,217,821","3,287,203,341","3,287,204,311","3,262,416,481",0,6,1105,148232
3,TB2R020100,Postpaid Revenue B2B,2025-03-19 11:39:49,20240102,20250228,0,"408,903,578","384,209,282","384,208,054","384,208,054",0,0,5,177,24603
4,TB3R000100,Broadband Revenue,2025-03-21 14:11:16,20240102,20250228,0,0,"1,573,227,163","1,573,118,602","1,573,088,544","1,560,922,362",0,5,1102,138718
5,TB4R000100,TVS Revenue,2025-03-14 13:59:20,20240102,20250228,0,0,"328,867,343","323,172,106","327,520,964","322,352,854",0,5,1104,106301


In [5]:
''' Current Month Subscription '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH']==curr_mth]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-26 04:50:14,20250324,20250324,0,0,"17,447,198","17,447,198","17,447,198","17,385,550",0,5,1104,1104
1,TB1S000600,Prepay Active Caller Subs,2025-03-16 04:50:51,20250314,20250314,0,"14,567,698","14,355,888","14,355,888","14,355,888","14,304,384",0,6,1105,1105
2,TB2S000500,Postpaid Active Subs,2025-03-26 05:01:42,20250325,20250325,0,"10,209,655","10,209,124","10,209,112","10,209,112","10,134,229",0,6,1105,1105
3,TB2S010500,Postpaid Active Subs B2C,2025-03-26 05:02:42,20250325,20250325,0,"9,023,166","9,022,923","9,022,911","9,022,911","8,967,767",0,6,1105,1105
4,TB2S020400,Postpaid Active Subs B2B,2025-03-26 05:00:52,20250325,20250325,0,"1,186,489","1,186,201","1,186,201","1,186,201",0,0,5,177,177
5,TB3S000500,Broadband Active Subs,2025-03-26 04:55:29,20250325,20250325,0,0,"3,122,190","3,122,190","3,122,190","3,122,190",0,5,1102,1102
6,TB4S000500,TVS Active Subs,2025-03-26 04:55:38,20250325,20250325,0,0,"1,154,454","1,154,454","1,154,454","1,154,439",0,5,1104,1104


In [6]:
''' Average Subscription per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-20 12:12:14,20240131,20250228,0,0,"17,662,374","17,662,181","17,662,181","17,600,486",0,5,1104,15456
1,TB1S000600,Prepay Active Caller Subs,2025-03-07 04:50:40,20240131,20250228,0,"14,441,024","14,342,605","14,362,024","14,362,051","14,312,460",0,6,1105,15470
2,TB2S000500,Postpaid Active Subs,2025-03-19 21:31:43,20240131,20250228,0,"10,021,687","9,921,762","9,922,029","9,922,053","9,849,626",0,6,1105,15470
3,TB2S010500,Postpaid Active Subs B2C,2025-03-19 21:32:10,20240131,20250228,0,"8,891,597","8,887,629","8,887,839","8,887,863","8,832,683",0,6,1105,15470
4,TB2S020400,Postpaid Active Subs B2B,2025-03-19 21:31:15,20240131,20250228,0,"1,130,090","1,034,133","1,034,190","1,034,190",0,0,5,177,2478
5,TB3S000500,Broadband Active Subs,2025-03-01 04:55:28,20240131,20250228,0,0,"3,053,895","3,055,196","3,055,196","3,055,841",0,5,1102,15412
6,TB4S000500,TVS Active Subs,2025-03-01 04:55:38,20240131,20250228,0,0,"1,220,146","1,219,989","1,219,989","1,307,223",0,5,8408,22760


In [7]:
''' METRIC Summary '''

src_t_rev_mth_df = chk_src_df.copy()
src_t_rev_mth_df = src_t_rev_mth_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

mod_col_list = src_t_rev_mth_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=10)}')


     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE  START_DAY   END_DAY                  P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-26 04:44:33   20240101  20250323                  0              5         1104   494592
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-26 04:41:32   20240101  20250323                  0              4          176    78848
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-26 04:43:02   20240101  20250323                  0              4          176    78848
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-26 04:50:14   20240131  20250324                  0              5         1104    16560
4   jutar11     TB1S000600   Prepay Active Caller Subs 2025-03-16 04:50:51   20240131  20250314        216,742,036              6         1105    16575
5   jutar11     TB2R000100            Postpaid Revenue 2025-03-26 04:20:14   20240102  

In [8]:
''' METRIC Current Month '''

# v_tm_key_mth = '202502'
curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_t_rev_mth_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {curr_mth}')

src_t_rev_mth_df = chk_src_df[src_t_rev_mth_df_cols].copy()
src_t_rev_mth_df = src_t_rev_mth_df.loc[src_t_rev_mth_df['TM_KEY_MTH']==curr_mth]
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_rev_mth_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202503

     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C              P              G              H             HH           CCAA CCAATT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-26 04:44:33     202503   20250301  20250323  0              0  1,853,638,095  1,853,629,953  1,853,629,953  1,845,870,444      0
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-26 04:41:32     202503   20250301  20250323  0              0    238,499,415    238,499,415    238,499,415              0      0
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-26 04:43:02     202503   20250301  20250323  0              0  1,615,138,680  1,615,130,538  1,615,130,538              0      0
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-26 04:50:14     202503   20250324  20250324  0              0     17,447,198     17,447,198     17,447,198     17,385,55

## Actual : Transaction
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [9]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    --WHERE METRIC_CD = 'TB3R000100' --TOL Revenue
    WHERE METRIC_CD IN (
        'TB1R000100' --'Prepay Revenue
        , 'TB2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         )
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Daily DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-03-27, 10:02:43

   -> Daily DataFrame : 158 rows, 12 columns

TDMDBPR : Disconnected


In [10]:
''' METRIC by Day '''

v_tm_key_day = 20250101
# src_t_rev_day_df = ['SRC', 'REMARK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT', 'LOAD_DATE']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

# src_t_rev_day_df = chk_src_df[src_t_rev_day_df_cols].copy()
src_t_rev_day_df = chk_src_df.copy()
src_t_rev_day_df = src_t_rev_day_df.loc[src_t_rev_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_t_rev_day_df = src_t_rev_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_t_rev_day_df.iloc[:, 4:11].columns.tolist()
for col in mod_col_list:
    src_t_rev_day_df[col] = src_t_rev_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_t_rev_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_t_rev_day_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_t_rev_day_df...

   -> DataFrame : 158 rows, 12 columns


In [11]:
''' TB1R000100 : Prepay Revenue '''

v_metric_cd = 'TB1R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB1R000100 : Prepay Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1R000100_day_df = chk_src_df.copy()
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['METRIC_CD']==v_metric_cd]
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1R000100_day_df = TB1R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1R000100_day_df[col] = TB1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1R000100_day_df


TB1R000100 : Prepay Revenue...

   -> v_metric_cd: TB1R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB1R000100,Prepay Revenue,20250301,2025-03-26 04:44:33,0,0,"98,889,500","98,888,516","98,888,516","98,429,657",0
1,jutar11,TB1R000100,Prepay Revenue,20250302,2025-03-26 04:44:33,0,0,"97,912,832","97,912,590","97,912,590","97,445,652",0
2,jutar11,TB1R000100,Prepay Revenue,20250303,2025-03-26 04:44:33,0,0,"97,338,888","97,338,362","97,338,362","96,903,957",0
3,jutar11,TB1R000100,Prepay Revenue,20250304,2025-03-26 04:44:33,0,0,"87,084,103","87,083,625","87,083,625","86,694,261",0
4,jutar11,TB1R000100,Prepay Revenue,20250305,2025-03-26 04:44:33,0,0,"88,600,987","88,600,830","88,600,830","88,245,728",0
5,jutar11,TB1R000100,Prepay Revenue,20250306,2025-03-26 04:44:33,0,0,"83,012,958","83,012,832","83,012,832","82,665,817",0
6,jutar11,TB1R000100,Prepay Revenue,20250307,2025-03-26 04:44:33,0,0,"84,162,402","84,161,169","84,161,169","83,814,116",0
7,jutar11,TB1R000100,Prepay Revenue,20250308,2025-03-26 04:44:33,0,0,"79,923,941","79,923,859","79,923,859","79,591,402",0
8,jutar11,TB1R000100,Prepay Revenue,20250309,2025-03-26 04:44:33,0,0,"77,196,629","77,196,578","77,196,578","76,882,443",0
9,jutar11,TB1R000100,Prepay Revenue,20250310,2025-03-26 04:44:33,0,0,"83,599,440","83,599,322","83,599,322","83,261,911",0


In [12]:
''' TB2R010100 : Postpaid Revenue B2C '''

v_metric_cd = 'TB2R010100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB2R010100 : Postpaid Revenue B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2R010100_day_df = chk_src_df.copy()
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['METRIC_CD']==v_metric_cd]
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2R010100_day_df = TB2R010100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2R010100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2R010100_day_df[col] = TB2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2R010100_day_df


TB2R010100 : Postpaid Revenue B2C...

   -> v_metric_cd: TB2R010100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB2R010100,Postpaid Revenue B2C,20250302,2025-03-26 04:21:45,0,"475,131,451","475,123,051","475,122,152","475,122,152","473,607,521",0
1,jutar11,TB2R010100,Postpaid Revenue B2C,20250306,2025-03-26 04:21:45,0,"279,710,998","279,705,332","279,705,332","279,705,332","278,402,695",0
2,jutar11,TB2R010100,Postpaid Revenue B2C,20250310,2025-03-26 04:21:45,0,"441,767,700","441,759,030","441,759,030","441,759,030","440,486,519",0
3,jutar11,TB2R010100,Postpaid Revenue B2C,20250313,2025-03-26 04:21:45,0,"388,887,467","388,878,026","388,877,577","388,877,577","387,739,742",0
4,jutar11,TB2R010100,Postpaid Revenue B2C,20250316,2025-03-26 04:21:45,0,"353,419,754","353,414,984","353,414,578","353,414,578","352,318,221",0
5,jutar11,TB2R010100,Postpaid Revenue B2C,20250319,2025-03-26 04:21:45,0,"351,783,068","351,774,294","351,774,294","351,774,294","350,611,425",0
6,jutar11,TB2R010100,Postpaid Revenue B2C,20250322,2025-03-26 04:21:45,0,"359,914,273","359,907,076","359,907,036","359,907,036","358,730,429",0
7,jutar11,TB2R010100,Postpaid Revenue B2C,20250325,2025-03-26 04:21:45,0,164,164,164,164,164,0


In [13]:
''' TB3R000100 : Broadband Revenue '''

v_metric_cd = 'TB3R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB3R000100 : Broadband Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD']==v_metric_cd]
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TB3R000100 : Broadband Revenue...

   -> v_metric_cd: TB3R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB3R000100,Broadband Revenue,20250302,2025-03-26 05:07:05,0,0,"218,022,091","218,022,091","218,022,091","218,057,466",0
1,salisa,TB3R000100,Broadband Revenue,20250306,2025-03-26 05:07:05,0,0,"143,302,592","143,302,592","143,302,592","143,302,592",0
2,salisa,TB3R000100,Broadband Revenue,20250310,2025-03-26 05:07:05,0,0,"232,927,431","232,927,431","232,927,431","232,952,693",0
3,salisa,TB3R000100,Broadband Revenue,20250313,2025-03-26 05:07:05,0,0,"173,112,878","173,112,878","173,112,878","173,112,878",0
4,salisa,TB3R000100,Broadband Revenue,20250316,2025-03-26 05:07:05,0,0,"174,114,474","174,114,474","174,114,474","174,117,818",0
5,salisa,TB3R000100,Broadband Revenue,20250319,2025-03-26 05:07:05,0,0,"166,812,264","166,812,264","166,812,264","166,812,798",0
6,salisa,TB3R000100,Broadband Revenue,20250322,2025-03-26 05:07:05,0,0,"164,416,665","164,416,665","164,416,665","164,445,271",0


In [14]:
''' TB4R000100 : TVS Revenue '''

v_metric_cd = 'TB4R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB4R000100 : TVS Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD']==v_metric_cd]
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TB4R000100 : TVS Revenue...

   -> v_metric_cd: TB4R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB4R000100,TVS Revenue,20250302,2025-03-19 14:24:58,0,0,"33,915,217","33,928,404","33,945,767",0,0
1,salisa,TB4R000100,TVS Revenue,20250306,2025-03-19 14:24:58,0,0,"9,508,034","9,599,873","9,605,668",0,0
2,salisa,TB4R000100,TVS Revenue,20250310,2025-03-19 14:24:58,0,0,"29,561,765","29,671,805","29,689,081",0,0
3,salisa,TB4R000100,TVS Revenue,20250313,2025-03-19 14:24:58,0,0,"21,702,020","21,763,584","21,771,609",0,0
4,salisa,TB4R000100,TVS Revenue,20250316,2025-03-19 14:24:58,0,0,"22,913,596","22,913,596","22,913,596",0,0
